In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Time Series Analysis\n",
    "\n",
    "This notebook implements time series analysis on the conversation dataset to analyze temporal patterns in message frequency, sentiment, and other metrics."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import required libraries\n",
    "import json\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from datetime import datetime\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from statsmodels.tsa.seasonal import seasonal_decompose\n",
    "from statsmodels.tsa.stattools import adfuller\n",
    "from statsmodels.graphics.tsaplots import plot_acf, plot_pacf\n",
    "from prophet import Prophet\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading and Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load the JSON data\n",
    "def load_data(file_path):\n",
    "    with open(file_path, 'r', encoding='utf-8') as f:\n",
    "        data = json.load(f)\n",
    "    return pd.DataFrame(data)\n",
    "\n",
    "# Load the dataset\n",
    "df = load_data('Parallel-Prod.AssistMessage.json')\n",
    "\n",
    "# Convert createdAt to datetime\n",
    "df['createdAt'] = pd.to_datetime(df['createdAt'].apply(lambda x: x['$date']))\n",
    "\n",
    "# Sort by timestamp\n",
    "df = df.sort_values('createdAt')\n",
    "\n",
    "# Create time-based features\n",
    "df['hour'] = df['createdAt'].dt.hour\n",
    "df['day'] = df['createdAt'].dt.day\n",
    "df['month'] = df['createdAt'].dt.month\n",
    "df['year'] = df['createdAt'].dt.year\n",
    "df['dayofweek'] = df['createdAt'].dt.dayofweek\n",
    "\n",
    "print(\"Dataset timespan:\")\n",
    "print(f\"Start: {df['createdAt'].min()}\")\n",
    "print(f\"End: {df['createdAt'].max()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Message Frequency Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Analyze message frequency\n",
    "def analyze_message_frequency():\n",
    "    # Daily message count\n",
    "    daily_messages = df.resample('D', on='createdAt').size()\n",
    "    \n    # Weekly message count\n",
    "    weekly_messages = df.resample('W', on='createdAt').size()\n",
    "    \n    # Monthly message count\n",
    "    monthly_messages = df.resample('M', on='createdAt').size()\n",
    "    \n    # Plot the results\n",
    "    fig, axes = plt.subplots(3, 1, figsize=(15, 12))\n",
    "    \n    # Daily\n",
    "    daily_messages.plot(ax=axes[0], title='Daily Message Frequency')\n",
    "    axes[0].set_xlabel('Date')\n",
    "    axes[0].set_ylabel('Number of Messages')\n",
    "    \n    # Weekly\n",
    "    weekly_messages.plot(ax=axes[1], title='Weekly Message Frequency')\n",
    "    axes[1].set_xlabel('Date')\n",
    "    axes[1].set_ylabel('Number of Messages')\n",
    "    \n    # Monthly\n",
    "    monthly_messages.plot(ax=axes[2], title='Monthly Message Frequency')\n",
    "    axes[2].set_xlabel('Date')\n",
    "    axes[2].set_ylabel('Number of Messages')\n",
    "    \n    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n    return daily_messages, weekly_messages, monthly_messages\n",
    "\n",
    "daily_messages, weekly_messages, monthly_messages = analyze_message_frequency()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Time Series Decomposition"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Perform time series decomposition\n",
    "def analyze_time_series_components(series, period):\n",
    "    # Perform decomposition\n",
    "    decomposition = seasonal_decompose(\n",
    "        series,\n",
    "        period=period,\n",
    "        model='additive'\n",
    "    )\n",
    "    \n    # Plot components\n",
    "    fig, axes = plt.subplots(4, 1, figsize=(15, 12))\n",
    "    \n    # Original\n",
    "    decomposition.observed.plot(ax=axes[0])\n",
    "    axes[0].set_title('Original Time Series')\n",
    "    \n    # Trend\n",
    "    decomposition.trend.plot(ax=axes[1])\n",
    "    axes[1].set_title('Trend')\n",
    "    \n    # Seasonal\n",
    "    decomposition.seasonal.plot(ax=axes[2])\n",
    "    axes[2].set_title('Seasonal')\n",
    "    \n    # Residual\n",
    "    decomposition.resid.plot(ax=axes[3])\n",
    "    axes[3].set_title('Residual')\n",
    "    \n    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n    return decomposition\n",
    "\n",
    "# Analyze weekly patterns\n",
    "weekly_decomposition = analyze_time_series_components(weekly_messages, period=52)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Pattern Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def analyze_patterns():\n",
    "    # Hourly patterns\n",
    "    hourly_pattern = df.groupby('hour').size()\n",
    "    \n    # Daily patterns\n",
    "    daily_pattern = df.groupby('dayofweek').size()\n",
    "    \n    # Monthly patterns\n",
    "    monthly_pattern = df.groupby('month').size()\n",
    "    \n    # Plot patterns\n",
    "    fig, axes = plt.subplots(3, 1, figsize=(15, 12))\n",
    "    \n    # Hourly\n",
    "    hourly_pattern.plot(kind='bar', ax=axes[0], title='Message Distribution by Hour')\n",
    "    axes[0].set_xlabel('Hour of Day')\n",
    "    axes[0].set_ylabel('Number of Messages')\n",
    "    \n    # Daily\n",
    "    daily_pattern.plot(kind='bar', ax=axes[1], title='Message Distribution by Day of Week')\n",
    "    axes[1].set_xlabel('Day of Week (0=Monday)')\n",
    "    axes[1].set_ylabel('Number of Messages')\n",
    "    \n    # Monthly\n",
    "    monthly_pattern.plot(kind='bar', ax=axes[2], title='Message Distribution by Month')\n",
    "    axes[2].set_xlabel('Month')\n",
    "    axes[2].set_ylabel('Number of Messages')\n",
    "    \n    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "analyze_patterns()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Time Series Forecasting using Prophet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def forecast_messages():\n",
    "    # Prepare data for Prophet\n",
    "    daily_df = daily_messages.reset_index()\n",
    "    daily_df.columns = ['ds', 'y']\n",
    "    \n    # Create and fit model\n",
    "    model = Prophet(\n",
    "        yearly_seasonality=True,\n",
    "        weekly_seasonality=True,\n",
    "        daily_seasonality=True\n",
    "    )\n",
    "    model.fit(daily_df)\n",
    "    \n    # Make future predictions\n",
    "    future = model.make_future_dataframe(periods=30)  # 30 days forecast\n",
    "    forecast = model.predict(future)\n",
    "    \n    # Plot forecast\n",
    "    fig = model.plot(forecast)\n",
    "    plt.title('Message Frequency Forecast')\n",
    "    plt.show()\n",
    "    \n    # Plot components\n",
    "    fig = model.plot_components(forecast)\n",
    "    plt.show()\n",
    "    \n    return model, forecast\n",
    "\n",
    "model, forecast = forecast_messages()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Interactive Visualization using Plotly"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def create_interactive_plots():\n",
    "    # Create interactive time series plot\n",
    "    fig = go.Figure()\n",
    "    \n    # Add daily messages\n",
    "    fig.add_trace(go.Scatter(\n",
    "        x=daily_messages.index,\n",
    "        y=daily_messages.values,\n",
    "        mode='lines',\n",
    "        name='Daily Messages'\n",
    "    ))\n",
    "    \n    # Add 7-day moving average\n",
    "    ma7 = daily_messages.rolling(window=7).mean()\n",
    "    fig.add_trace(go.Scatter(\n",
    "        x=ma7.index,\n",
    "        y=ma7.values,\n",
    "        mode='lines',\n",
    "        name='7-day Moving Average',\n",
    "        line=dict(dash='dash')\n",
    "    ))\n",
    "    \n    fig.update_layout(\n",
    "        title='Interactive Time Series Plot',\n",
    "        xaxis_title='Date',\n",
    "        yaxis_title='Number of Messages',\n",
    "        hovermode='x unified'\n",
    "    )\n",
    "    \n    fig.show()\n",
    "\n",
    "create_interactive_plots()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}